In [45]:
import gurobipy as gp
from gurobipy import GRB
import random as rd
import requests

In [46]:
A = [0, 1, 2,3,4,5,6]
T = [0,1,2]
V = [ 'walk',  'car']
shared = [False, False, True]
Available = [1000, 2, 2]

In [47]:
Q = 15
Budget = 10
n_passengers = 1
cost_rate_v = {'walk': 0, 'bike': 1.0, 'car': 2.0}
travel_time_v = {'walk': 1, 'bike': 0.5, 'car': 0.25}
fixed_cost = 5

coordinates = {
    0: (43.651070, -79.347015),  # Toronto City Hall (Depot)
    1: (43.662892, -79.395656),  # University of Toronto - St. George Campus
    2: (43.648690, -79.385439),  # CN Tower
    3: (43.665860, -79.383990),  # Royal Ontario Museum
    4: (43.657305, -79.378338),  # Queen's Park
    5: (43.651900, -79.377290),  # Toronto Public Library - Toronto Reference Library
    6: (43.645380, -79.380243),  # Rogers Centre
}

def fetch_osrm_matrix(coords, profile='car'):
    node_ids = sorted(coords.keys())
    coords_str = ";".join(f"{coords[i][1]},{coords[i][0]}" for i in node_ids)
    url = f"http://router.project-osrm.org/table/v1/{profile}/{coords_str}?annotations=distance,duration"
    response = requests.get(url)
    data = response.json()
    
    if 'distances' not in data or 'durations' not in data:
        raise Exception(f"OSRM API error or rate limit exceeded: {data}")
    
    distances = data['distances']
    durations = data['durations']
    dist_dict = {}
    dur_dict = {}
    
    n = len(node_ids)
    for i_idx in range(n):
        for j_idx in range(n):
            i = node_ids[i_idx]
            j = node_ids[j_idx]
            dist_dict[(i,j)] = distances[i_idx][j_idx]
            dur_dict[(i,j)] = durations[i_idx][j_idx]
    return dist_dict, dur_dict

distance_ij_car, duration_ij_car = fetch_osrm_matrix(coordinates, profile='car')
distance_ij_walk, duration_ij_walk = fetch_osrm_matrix(coordinates, profile='foot')

distance_ij = {}
t_ij_v = {}

for v in V:
    if v == 'car':
        dist = distance_ij_car
        dur = duration_ij_car
    if v == 'bike':
        dist = distance_ij_car
        dur = duration_ij_car
    elif v == 'walk':
        dist = distance_ij_walk
        dur = duration_ij_walk
    else:
        dist = {}
        dur = {}

    for i in A:
        for j in A:
            distance_ij[(i,j)] = dist.get((i,j), 0)
            t_ij_v[(v,i,j)] = dur.get((i,j), 0) / 60

print(t_ij_v)

tv_j = {
    0: 0,   # Depot usually has 0 service time
    1: 1,
    2: 2,
    3: 1,
    4: 1.5,
    5: 2,
    6: 1,
    7: 1.5,
    8: 1,
    9: 2
}


open_windows = {
    0: (8, 18),   # Depot: Open all day
    1: (9, 12),
    2: (10, 14),
    3: (11, 15),
    4: (13, 17),
    5: (9, 16),
    6: (10, 13),
    7: (14, 18),
    8: (9, 11),
    9: (15, 19)
}

for i in A:
    print(f"open_hours({i}) = {open_windows[i][0]} to {open_windows[i][1]}")


{('walk', 0, 0): 0.0, ('walk', 0, 1): 12.391666666666667, ('walk', 0, 2): 6.3516666666666675, ('walk', 0, 3): 9.101666666666667, ('walk', 0, 4): 6.883333333333334, ('walk', 0, 5): 5.38, ('walk', 0, 6): 5.681666666666667, ('walk', 1, 0): 12.146666666666667, ('walk', 1, 1): 0.0, ('walk', 1, 2): 5.72, ('walk', 1, 3): 5.361666666666666, ('walk', 1, 4): 6.61, ('walk', 1, 5): 7.9, ('walk', 1, 6): 7.176666666666667, ('walk', 2, 0): 6.426666666666667, ('walk', 2, 1): 6.158333333333333, ('walk', 2, 2): 0.0, ('walk', 2, 3): 5.846666666666667, ('walk', 2, 4): 3.82, ('walk', 2, 5): 2.8333333333333335, ('walk', 2, 6): 1.4566666666666668, ('walk', 3, 0): 9.126666666666667, ('walk', 3, 1): 4.24, ('walk', 3, 2): 5.165, ('walk', 3, 3): 0.0, ('walk', 3, 4): 4.126666666666667, ('walk', 3, 5): 4.791666666666667, ('walk', 3, 6): 6.025, ('walk', 4, 0): 7.136666666666667, ('walk', 4, 1): 7.136666666666667, ('walk', 4, 2): 4.821666666666667, ('walk', 4, 3): 4.043333333333333, ('walk', 4, 4): 0.0, ('walk', 4, 

In [48]:
m = gp.Model("model")
# Variables
x = m.addVars(A, A, V, T, vtype=GRB.BINARY, name="x")

m.setObjective(gp.quicksum(x[i, j, v, t]  for i in A for j in A for v in V for t in T), GRB.MAXIMIZE) 
# Constraints

m.addConstrs(gp.quicksum(x[i, i, v, t] for t in T for v in V) == 0 for i in A)
m.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for v in V) == gp.quicksum(x[j, i, v, t] for i in A for v in V) for j in A for t in T)
m.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for t in T for v in V) <= 1 for j in A[1:])

m.addConstrs(gp.quicksum(x[0, j, v, t] for j in A[1:] for v in V) <= 1 for t in T )

m.addConstrs(gp.quicksum(x[i, j, v, t] * (tv_j[j] + t_ij_v[v, i, j]) for i in A for j in A for v in V ) <= Q for t in T )

#Miller-Tucker-Zemlin formulation Subtour Elimination Constraints
q = [rd.randint(3, 3) for i in range(len(A)-1)]  # generate the random vector
u = m.addVars([i for i in range(len(A)-1)],V, vtype=GRB.CONTINUOUS, name="u")
for v in V: u[0, v].LB = 0
for v in V: u[0, v].UB = 0
P = [i for i in range(len(A)-1)]
for v in V :
    for i in P :
            
    #             u[i].LB = q[i]
        u[i, v].LB = min([(q[i]+(tv_j[i] + t_ij_v[v, j,i])) for j in P for t in T])
        u[i, v].UB = Q
    #     print([(q[i]+tm[j][i])* xijt[i,j,t] for i in P for j in P for t in T])
        m.addConstrs( u[i, v] - u[j, v] + Q * x[i, j, v, t] <= Q - q[j] for i in P for j in P for t in T if j != 0 )



In [49]:
m.optimize()
if m.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for v in V:
        for i in A:
            for j in A:
                for t in T:
                    if x[i, j, v, t].x > 0.5:
                        print(f"Vehicle {v} travels from {i} to {j} at day {t}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1120 rows, 306 columns and 3984 nonzeros
Model fingerprint: 0x73d18784
Variable types: 12 continuous, 294 integer (294 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 957 rows and 77 columns
Presolve time: 0.01s
Presolved: 163 rows, 229 columns, 1255 nonzeros
Variable types: 10 continuous, 219 integer (219 binary)
Found heuristic solution: objective 8.0000000

Root relaxation: objective 9.000000e+00, 79 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0    

In [50]:
m1 = gp.Model("model tw")

# Variables
x = m1.addVars(A, A, V, T, vtype=GRB.BINARY, name="x")
t_ = m1.addVars(A,V, T, vtype=GRB.CONTINUOUS, name="t")
y = m1.addVars(T, vtype=GRB.BINARY, name='y')
b = m1.addVars(T, vtype=GRB.CONTINUOUS, name='b')

# Obj Function
m1.setObjective(gp.quicksum(x[i, j, v, t] * (1 / t_ij_v[v, i, j]) for i in A for j in A for v in V for t in T if i != j and t_ij_v[v, i, j] > 1e-6), GRB.MAXIMIZE)

# Constraints
m1.addConstrs(gp.quicksum(x[i, i, v, t] for t in T for v in V) == 0 for i in A)
m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for v in V) == gp.quicksum(x[j, i, v, t] for i in A for v in V) for j in A for t in T)
m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for t in T for v in V) <= 1 for j in A[1:])
m1.addConstrs(gp.quicksum(x[0, j, v, t] for j in A[1:] for v in V) == y[t] for t in T)
m1.addConstrs(gp.quicksum(x[i, j, v, t] * (tv_j[j] + t_ij_v[v, i, j]) for i in A for j in A for v in V ) <= Q for t in T )

m1.addConstrs(y[t] <= y[t+1] for t in T[:-1])  # Ensure y is non-decreasing
m1.addConstr(y[0]==1)  # Ensure y is non-increasing

# Time window constraints
m1.addConstrs(t_[i, v, t] >= open_windows[i][0] for i in A for v in V for t in T)
m1.addConstrs(t_[i, v, t] <= open_windows[i][1] for i in A for v in V for t in T)

#dialy budget constraints
m1.addConstrs(gp.quicksum((n_passengers if shared[V.index(v)] == False else 1)*x[i, j, v, t] * cost_rate_v[v]* t_ij_v[v, i, j] for i in A for j in A for v in V ) <= b[t] for t in T)
m1.addConstr(gp.quicksum(b[t] for t in T) + fixed_cost<= Budget)

#m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for j in A) <= Available[V.index(v)] for v in V for t in T)
m1.addConstrs( x[i, j, v, t] * n_passengers  <= Available[V.index(v)] for i in A for j in A for v in V for t in T)
m1.addConstrs(t_[i, v, t] + tv_j[i] + t_ij_v[v, i, j] - 1000 * (1 - x[i, j, v, t]) <= t_[j, v, t] for v in V for i in A for j in A[1:] for t in T   )

#Miller-Tucker-Zemlin formulation Subtour Elimination Constraints
q = [rd.randint(3, 3) for i in range(len(A)-1)]  # generate the random vector
u = m1.addVars([i for i in range(len(A)-1)],V, vtype=GRB.CONTINUOUS, name="u")
for v in V: u[0, v].LB = 0
for v in V: u[0, v].UB = 0
P = [i for i in range(len(A)-1)]
for v in V :
    for i in P :
            
    #             u[i].LB = q[i]
        u[i, v].LB = min([(q[i]+(tv_j[i] + t_ij_v[v, j,i])) for j in P for t in T])
        u[i, v].UB = Q
    #     print([(q[i]+tm[j][i])* xijt[i,j,t] for i in P for j in P for t in T])
        m1.addConstrs( u[i, v] - u[j, v] + Q * x[i, j, v, t] <= Q - q[j] for i in P for j in P for t in T if j != 0 )


In [51]:
m1.optimize()
if m1.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for t in T:
        for i in A:
            for j in A:
                for v in V:
                    if x[i, j, v, t].x > 0.5:
                        print(f"Vehicle {v} travels from {i} to {j} at day {t}")
                        print(f"Cost for vehicle {v} is {(n_passengers if shared[V.index(v)] == False else 1)*cost_rate_v[v] * t_ij_v[v, i, j]}")
                        print(f"Time at {i} is {t_[i, v, t].x}")
                        print(f"Time at {j} is {t_[j, v, t].x}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1757 rows, 354 columns and 5060 nonzeros
Model fingerprint: 0x7ddf76d9
Variable types: 57 continuous, 297 integer (297 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e-02, 7e-01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 0.4419124
Presolve removed 1560 rows and 155 columns
Presolve time: 0.01s
Presolved: 197 rows, 199 columns, 2087 nonzeros
Variable types: 31 continuous, 168 integer (168 binary)
Found heuristic solution: objective 0.6358988

Root relaxation: objective 1.793053e+00, 76 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

In [52]:
if m1.status == GRB.OPTIMAL:
    print("Optimal solution found:\n")

    for t in T:
        print(f"--- Trips for Day {t} ---")
        trips_found = False
        for i in A:
            for j in A:
                for v in V:
                    if i != j and x[i, j, v, t].x > 0.5:
                        trips_found = True
                        print(f"Vehicle [{v}] from {i} to {j}")
        if not trips_found:
            print("No trips on this day.")
            
        print("\n")

Optimal solution found:

--- Trips for Day 0 ---
Vehicle [walk] from 0 to 5
Vehicle [walk] from 5 to 0


--- Trips for Day 1 ---
Vehicle [car] from 0 to 6
Vehicle [walk] from 2 to 0
Vehicle [walk] from 6 to 2


--- Trips for Day 2 ---
Vehicle [car] from 0 to 3
Vehicle [walk] from 3 to 4
Vehicle [walk] from 4 to 0




In [53]:
if m1.status == GRB.OPTIMAL:
    mymap = folium.Map(location=coordinates[0], zoom_start=13)

    for node in A:
        folium.Marker(
            location=coordinates[node],
            popup=f"Node {node}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(mymap)

    day_colors = generate_day_colors(len(T))

    for t in T:
        route_color = day_colors[t]
        for v in V:
            for i in A:
                for j in A:
                    if i != j and x[i, j, v, t].X > 0.5:
                        folium.PolyLine(
                            locations=[coordinates[i], coordinates[j]],
                            color=route_color,
                            weight=5,
                            opacity=0.7,
                            popup=f"Day: {t}, Vehicle: {v}, From {i} to {j}"
                        ).add_to(mymap)

    mymap.save("routing.html")
    print("Map saved as routing.html")

else:
    print("Model not solved optimally. Cannot plot routes.")

Map saved as routing.html
